# Unzip bbox files in Azure Blob Storage

In [1]:
from azure.storage.blob import BlobClient
import tarfile
import os

compute_data_folder = "../../data/"
zip_files = ["Segments_mat_gt_bb_S11.tgz", 
             "Segments_mat_gt_bb_S5.tgz", "Segments_mat_gt_bb_S6.tgz", "Segments_mat_gt_bb_S7.tgz",
            "Segments_mat_gt_bb_S8.tgz", "Segments_mat_gt_bb_S9.tgz"]
#zip_files = ["Segments_mat_gt_bb_S1.tgz"]
conn_str = "DefaultEndpointsProtocol=https;AccountName=nyposeestimation;AccountKey=Uf7Ket/F5XqlnHJx3vbJ/HDJNNzGpnUIuNo2rCg4CxAVHgF3+V574b2Siw6EM0Ef5+Rw5aWUd2k++AStXDZ8fQ==;EndpointSuffix=core.windows.net"
container_name = "human36m"
blob_data_folder = "data/"

ModuleNotFoundError: No module named 'azure'

In [ ]:
for zip_file in zip_files:
    # Get blob client
    blob_client = BlobClient.from_connection_string(conn_str=conn_str, 
                                             container_name=container_name, 
                                             blob_name=blob_data_folder + zip_file)
    # Save file into compute instance
    print("File: ", zip_file)
    print("Starting to save zip file into compute instance...")
    with open(compute_data_folder + zip_file, "wb+") as blob:
        blob_data = blob_client.download_blob()
        blob_data.readinto(blob)
    print("Finished saving zip file into compute instance.")
    
    # Unzip file in compute instance
    print("Starting to unzip file in compute instance...")
    unzipped_folder = zip_file[0:-4]
    unzipped_folder_prefix = compute_data_folder + unzipped_folder
    file = tarfile.open(compute_data_folder + zip_file)
    file.extractall(unzipped_folder_prefix)
    file.close()
    print("Finished unzipping file in compute instance.")
    
    # Upload bboxes back to azure storage
    print("Getting list of bboxes mat files")
    bboxes = os.listdir(unzipped_folder_prefix + "/" + unzipped_folder[19:] + "/MySegmentsMat/ground_truth_bb")
    # print(bboxes) # for double checking that only the first two are extraneous files, or not
    
    print("Starting to upload individual bboxes back to azure...")
        # For efficiency, store these path prefixes
    blob_new_folder = blob_data_folder + unzipped_folder + "/"
    compute_video_prefix = unzipped_folder_prefix + "/" + unzipped_folder[19:] + "/MySegmentsMat/ground_truth_bb/"
    for bbox in bboxes:
        blob = BlobClient.from_connection_string(conn_str=conn_str, 
                                                 container_name=container_name, 
                                                 blob_name=blob_new_folder + bbox)
        # blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_folder_prefix + zip_files[0][0:-4] + "/" + video)

        
        with open(file=compute_video_prefix + bbox, mode="rb") as data:
            blob.upload_blob(data)
    print("Done uploading ", zip_file, " to Azure.")
    

# Utils

### OpenCV Vocabulary
TCHW: Time (t), Channel (RGB), H (Height), W (Width)

### Directory Structure:

---- Videos_S11/

    ---- Videos/
        ----[].mp4
        ----[].mp4

In [6]:
import numpy as np
#import torch
#import matplotlib.pyplot as plt
#import torchvision.transforms.functional as F
#from torchvision.io import read_video # extract frames
import os
import h5py
#from torchvision.utils import save_image
#import torchvision.transforms as T
import cv2

## Post-process frames

In [14]:
"""plt.rcParams["savefig.bbox"] = "tight"
# sphinx_gallery_thumbnail_number = 2

def plot(imgs, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0])
    _, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            img = F.to_pil_image(img.to("cpu"))
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    plt.tight_layout()
    plt.show()"""

# get frames from video path
def get_frames(video_path):
    frames, _, _ = read_video(video_path, output_format="TCHW", pts_unit="sec")
    return frames

# make frame suitable for inference
def make_frame_suitable(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    return frame

def frame_resize_to_nparr(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    return np.array(frame)

def frame_to_tensor(frame):
    transform = T.ToPILImage()
    frame = transform(frame)
    frame = frame.resize((256, 256)) # resize to 256 x 256
    toTensor = T.ToTensor()
    return toTensor(frame)

# get list of all subjects
def get_subjects():
    video_folders = os.listdir(compute_data_folder)[4:]
    return video_folders

def get_video_paths(subject):
    video_paths = os.listdir(compute_data_folder + subject + "/videos/")[2:]
    return video_paths

## Pre-process frames (tasks split to increase efficiency)

h36m

    +-- training
    
        +-- subject
        
            +-- frames
            
                +-- activity
                
                    +-- frame*.png

In [16]:
# CREATE DIRECTORY STRUCTURE
subjects = get_subjects()

# for each subject
for subject in subjects:
    video_paths = get_video_paths(subject)
    path_prefix = compute_data_folder + subject
    
    save_prefix = compute_data_folder + "h36m_final/training/" + subject + "/frames/"
    
    print("Processing subject ", subject)
    
    # for each video (i.e. activity), get list of frames
    for j in range(len(video_paths)):
        # print("----video ", (j+1) / len(video_paths))
        
        # get activity
        activity = video_paths[j][:-4]
        
        # path prefix for video
        save_prefix_vid = save_prefix + activity
        
        if not os.path.isdir(save_prefix_vid): # create directory structure is not present
            os.makedirs(save_prefix_vid)

Processing subject  S1
Processing subject  S11
Processing subject  S5
Processing subject  S6
Processing subject  S7
Processing subject  S8
Processing subject  S9


In [ ]:
subjects = get_subjects()
subjects

In [12]:
video_paths = get_video_paths("S1")
for i in range(len(video_paths)):
    if video_paths[i] == "SittingDown.55011271.mp4":
        print(i)
        break
video_paths[70:]

69


['SittingDown.58860488.mp4',
 'SittingDown.60457274.mp4',
 'Smoking 1.54138969.mp4',
 'Smoking 1.55011271.mp4',
 'Smoking 1.58860488.mp4',
 'Smoking 1.60457274.mp4',
 'Smoking.54138969.mp4',
 'Smoking.55011271.mp4',
 'Smoking.58860488.mp4',
 'Smoking.60457274.mp4',
 'TakingPhoto 1.54138969.mp4',
 'TakingPhoto 1.55011271.mp4',
 'TakingPhoto 1.58860488.mp4',
 'TakingPhoto 1.60457274.mp4',
 'TakingPhoto.54138969.mp4',
 'TakingPhoto.55011271.mp4',
 'TakingPhoto.58860488.mp4',
 'TakingPhoto.60457274.mp4',
 'Waiting 1.54138969.mp4',
 'Waiting 1.55011271.mp4',
 'Waiting 1.58860488.mp4',
 'Waiting 1.60457274.mp4',
 'Waiting.54138969.mp4',
 'Waiting.55011271.mp4',
 'Waiting.58860488.mp4',
 'Waiting.60457274.mp4',
 'Walking 1.54138969.mp4',
 'Walking 1.55011271.mp4',
 'Walking 1.58860488.mp4',
 'Walking 1.60457274.mp4',
 'Walking.54138969.mp4',
 'Walking.55011271.mp4',
 'Walking.58860488.mp4',
 'Walking.60457274.mp4',
 'WalkingDog 1.54138969.mp4',
 'WalkingDog 1.55011271.mp4',
 'WalkingDog 1.588

In [4]:
subjects = get_subjects()
#subjects = ['S1', 'S11', 'S5']

# for each subject
for subject in subjects:
    video_paths = get_video_paths(subject)
    path_prefix = compute_data_folder + subject
    
    save_prefix = compute_data_folder + "h36m/training/" + subject + "/frames/"
    
    print("Processing subject ", subject)
    
    # for each video (i.e. activity), get list of frames
    
    for j in range(len(video_paths)):
        #if j % 3 == 0:
        #    print("----video ", (j+1) / len(video_paths))
        
        video_path = path_prefix + "/videos/" + video_paths[j]
        activity = video_paths[j][:-4]
        
        save_prefix_vid = save_prefix + activity + "/"
        
        # get frames for this video
        vidcap = cv2.VideoCapture(video_path)
        i = 0
        while True:
            success,image = vidcap.read()
            if not success:
                break
            image = frame_resize_to_nparr(image)
            cv2.imwrite(save_prefix_vid + ("frame%04d.png" % i), image)     # save frame as JPEG file
            i += 1

Processing subject  S1


KeyboardInterrupt: 

## Method 2

In [ ]:
# process bounding boxes of one video
def get_masks_of_video(bbox_path):
    with h5py.File(bbox_path, 'r') as f:
        masks = np.array(f.get("Masks"))
        refs = np.array(f.get("#refs#"))
        
        return masks[:,0]
        
        #print(refs)
        """ thismask = masks[:,0][0]
        frame = np.array(f[thismask])
        print(frame.shape)"""
        #np.savetxt("mask_example.csv", frame, delimiter = ",")

        # for each frame
        """for mask in masks[:,0]:
            frame = np.array(f[mask])
            print(frame)"""
        #print()
        #print(refs.shape)
        
def get_masked_frame(bbox_path, mask):
    with h5py.File(bbox_path, 'r') as f:
        return np.array(f[mask])

In [ ]:
subjects = get_subjects()
subjects = ["S1"]

# for each subject
for subject in subjects:
    video_paths = get_video_paths(subject)
    bbox_paths = [(i[:-4] + ".mat") for i in video_paths]
    
    path_prefix = compute_data_folder + subject
    # for each video, get list of frames and corresponding masks
    for j in range(len(video_paths)):
        video_path = path_prefix + "/videos/" + video_paths[j]
        bbox_path = path_prefix + "/bboxes/" + bbox_paths[j]
        frames = get_frames(video_path)
        masks = get_masks_of_video(bbox_path)
        
        print("# Frames: ", len(frames))
        print("# Masks: ", len(masks))
        
        pass
        # for each frame
        for k in range(len(masks)):
            
            # remove this later
            # if (k % 100 == 0):
            mask = masks[k]
            frame = frames[k]
            masked_frame = get_masked_frame(bbox_path, mask).T
            #print("Frame shape: ", frame.shape)
            #print("mask shape: ", masked_frame.shape)

            masked_frame = frame * masked_frame
            plot(torch.stack([masked_frame]))
            
        # find frames whose bounding box frames are in the same place
            

    #frames = get_frames(video_path)
    #process_bbox(compute_data_folder + subject + "/bboxes/" + bbox_paths[0])

In [ ]:
# get frames of one video
video_path = "../../data/S9/videos/Sitting 1.58860488.mp4"
frames = get_frames(video_path)

In [ ]:
print(frames[25].shape)

In [ ]:
img1_batch = torch.stack([frames[0], frames[10], frames[20]])
img2_batch = torch.stack([frames[20], frames[25]])

plot(torch.stack([frames[1100]]))

# Inference

In [ ]:
from pretrained_model import Predictor
import torchvision.transforms as T

In [ ]:
device = torch.device('cuda:0')
pred = Predictor(batch_size=1, num_parts=18, device=device, template_path='../../template.json',
                 anchors_path='../../anchor_points.json')
pred.load_checkpoint('../../checkpoint.tar')

In [ ]:
# choose one frame
frame_example = frames[1100]
frame_example = make_frame_suitable(frame_example)

In [ ]:
prediction = pred.predict(frame_example)

# Visualize

In [ ]:
transformed_anchors = [0,0,0]
warped_heatmaps, transformed_anchors[0], transformed_anchors[1], transformed_anchors[2] = prediction

In [ ]:
warped_heatmaps_cpu = warped_heatmaps.squeeze().cpu()
warped_heatmaps_cpu_numpy = warped_heatmaps_cpu.detach().numpy()

In [ ]:
warped_heatmaps_cpu_numpy = warped_heatmaps_cpu.detach().numpy()
import matplotlib.pyplot as plt
import numpy as np

warped_heatmaps_cpu_numpy_sum = np.sum(warped_heatmaps_cpu_numpy, 0)

plt.imshow(warped_heatmaps_cpu_numpy_sum)
plt.show()

## Show Original Template

In [ ]:
template_nums = pred.template.squeeze(0).cpu().detach().numpy()
templates_sum = np.sum(template_nums, 0)

plt.imshow(templates_sum)
plt.show()